In [1]:
import os
import pandas as pd
import numpy as np
from monthly_dividend import YFfetcher, CoveredCallbacktester, GBM, evaluate_portfolio
from monthly_dividend import simulate_withdrawal_etf_only, simulate_withdrawal_etf_cash
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import warnings
from hmmlearn.hmm import GaussianHMM
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
import portfolio_pipeline as pp
import matplotlib
import matplotlib.dates as mdates
from matplotlib.patches import Patch
import itertools

In [2]:
os.chdir('C:/Users/user/Desktop/대학원/수업/2025-1/핀테크빅데이터분석심화/분석결과')

In [3]:
warnings.filterwarnings("ignore")

In [6]:
# 시작점: 2014.01.01~
SPY = YFfetcher.get_price_history("SPY", "2014-01-01", "2025-06-01")   # 1993.01.29~
QQQ = YFfetcher.get_price_history("QQQ", "2014-01-01", "2025-06-01")   # 2012.02.24~
SCHD = YFfetcher.get_price_history("SCHD", "2014-01-01", "2025-06-01")   # 2012.02.24~
XYLD = YFfetcher.get_price_history("XYLD", "2014-01-01", "2025-06-01")   # 2012.02.24~
QYLD = YFfetcher.get_price_history("QYLD", "2014-01-01", "2025-06-01")   # 2012.02.24~

# GOVT = YFfetcher.get_price_history("GOVT", "2014-01-01", "2025-06-01")   # 2012.02.24~
SHY = YFfetcher.get_price_history("SHY", "2014-01-01", "2025-06-01")   # 2012.02.24~

etf_dfs = {
    "SPY": SPY,
    "QQQ": QQQ,
    #"SCHD": SCHD,
    "XYLD": XYLD,
    "QYLD": QYLD,
    #"GOVT": GOVT,
    "SHY": SHY,
}

for ticker, df in etf_dfs.items():
    df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    df.set_index('Date', inplace=True)
    etf_dfs[ticker] = df[['Close']].rename(columns={'Close': ticker})

price_df = pd.concat(etf_dfs.values(), axis=1, join='inner')  

# 시작점: 2014.01.01~
SPY = YFfetcher.get_price_history("SPY", "2014-01-01", "2025-06-01")   # 1993.01.29~
QQQ = YFfetcher.get_price_history("QQQ", "2014-01-01", "2025-06-01")   # 2012.02.24~
SCHD = YFfetcher.get_price_history("SCHD", "2014-01-01", "2025-06-01")   # 2012.02.24~
XYLD = YFfetcher.get_price_history("XYLD", "2014-01-01", "2025-06-01")   # 2012.02.24~
QYLD = YFfetcher.get_price_history("QYLD", "2014-01-01", "2025-06-01")   # 2012.02.24~

# GOVT = YFfetcher.get_price_history("GOVT", "2014-01-01", "2025-06-01")   # 2012.02.24~
SHY = YFfetcher.get_price_history("SHY", "2014-01-01", "2025-06-01")   # 2012.02.24~

etf_dfs = {
    "SPY": SPY,
    "QQQ": QQQ,
    #"SCHD": SCHD,
    "XYLD": XYLD,
    "QYLD": QYLD,
    #"GOVT": GOVT,
    "SHY": SHY,
}

for ticker, df in etf_dfs.items():
    df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    df.set_index('Date', inplace=True)
    etf_dfs[ticker] = df[['Adj Close']].rename(columns={'Adj Close': ticker})

adj_price_df = pd.concat(etf_dfs.values(), axis=1, join='inner')  


# 1. ETF별 배당 정보 수집
SPY_div = YFfetcher.get_dividends("SPY", "2014-01-01", "2025-04-15")  
QQQ_div = YFfetcher.get_dividends("QQQ", "2014-01-01", "2025-04-15")   
SCHD_div = YFfetcher.get_dividends("SCHD", "2014-01-01", "2025-04-15")  
XYLD_div = YFfetcher.get_dividends("XYLD", "2014-01-01", "2025-04-15")  
QYLD_div = YFfetcher.get_dividends("QYLD", "2014-01-01", "2025-04-15")   
SHY_div = YFfetcher.get_dividends("SHY", "2014-01-01", "2025-04-15")   

etf_divs = {
    'SPY': SPY_div,
    'QQQ': QQQ_div,
    #'SCHD': SCHD_div,
    'XYLD': XYLD_div,
    'QYLD': QYLD_div,
    'SHY': SHY_div
}


# 2. 모든 배당을 하나의 DataFrame으로 합치기
div_records = []

for etf, df in etf_divs.items():
    temp = df.copy()
    temp['ETF'] = etf
    div_records.append(temp)

div_raw = pd.concat(div_records)

# 3. pivot: 일자별 ETF별 배당금
dividend_df = div_raw.pivot_table(index='Date', columns='ETF', values='Dividend', aggfunc='sum').fillna(0.0)
dividend_df.index = pd.to_datetime(dividend_df.index).normalize()

# 시뮬레이션할 일별 가격 인덱스와 align
dividend_df.index = pd.to_datetime(dividend_df.index).normalize()
dividend_df = dividend_df.reindex(price_df.index).fillna(0.0)


## HMM 추정용 데이터
# 시작점: 2014.01.01~
SPY = YFfetcher.get_price_history("SPY", "2013-12-31", "2025-06-01")   # 1993.01.29~
# QQQ = YFfetcher.get_price_history("QQQ", "2013-12-31", "2025-06-01")   # 2012.02.24~
VIX = YFfetcher.get_price_history("^VIX", "2013-12-31", "2025-06-01")

etf_dfs = {
    "SPY": SPY,
    #"QQQ": QQQ,
    "VIX": VIX
}

for ticker, df in etf_dfs.items():
    df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    df.set_index('Date', inplace=True)
    etf_dfs[ticker] = df[['Adj Close']].rename(columns={'Adj Close': ticker})

hmm_df = pd.concat(etf_dfs.values(), axis=1, join='inner')  

def compute_log_returns(df: pd.DataFrame) -> pd.DataFrame:
    """Adj Close 기준 수익률 계산 (HMM 입력용)"""
    logrets = np.log(df / df.shift(1)).dropna()
    return logrets

hmm_df = compute_log_returns(hmm_df)

In [7]:
SPY = YFfetcher.get_price_history("SPY", "2013-12-31", "2025-06-01")   # 1993.01.29~
VIX = YFfetcher.get_price_history("^VIX", "2013-12-31", "2025-06-01")

etf_dfs = {
    "SPY": SPY,
    #"QQQ": QQQ,
    "VIX": VIX
}

for ticker, df in etf_dfs.items():
    df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    df.set_index('Date', inplace=True)
    etf_dfs[ticker] = df[['Adj Close']].rename(columns={'Adj Close': ticker})

hmm_df2 = pd.concat(etf_dfs.values(), axis=1, join='inner')  

In [8]:
# 무위험 수익률
#irx = yf.download("^IRX", start="2014-01-01", end="2025-06-01", interval="1d") 
irx = YFfetcher.get_price_history("^IRX", "2017-01-01", "2025-06-01")   # 1993.01.29~
irx['Date'] = pd.to_datetime(irx['Date']).dt.normalize()
irx = irx.fillna(method='ffill')
irx = irx.groupby([irx['Date'].dt.year, irx['Date'].dt.month]).head(1)
#irx = irx.reindex(price_df.index).fillna(method='ffill')

irx_rates = irx['Close'].values / 100  # IRX 수익률: % 단위를 소수로 내변환
monthly_rf = (1 + irx_rates) ** (1/12) - 1       # 연 단리 → 월 복리 수익률 (IRX는 연 단리수익률) 

In [12]:
# 설정값
withdrawals = [800, 1600, 2400]
target_cvars = [0.02, 0.03, 0.04, 0.05]
methods = [("CVaR_L2", True), ("CVaR_L2", False), ("6040", False)]

# 결과 저장 리스트
result_rows = []

# 반복 실행
for wd, tc, (method, use_hmm) in itertools.product(withdrawals, target_cvars, methods):
    cfg = pp.BacktestConfig(monthly_withdraw=wd, target_cvar=tc)
    res = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df,
                      use_hmm=use_hmm, method=method, cfg=cfg)
    
    metrics = evaluate_portfolio(
        res['total_value'], 
        risk_free_rate=monthly_rf[1:len(res)], 
        periods_per_year=12
    )

    result_rows.append({
        "Withdraw": wd,
        "Target_CVaR": tc,
        "Method": method,
        "HMM": use_hmm,
        "Total Return": metrics["total_return"],
        "CAGR": metrics["cagr"],
        "Volatility": metrics["annualized_volatility"],
        "Sharpe": metrics["sharpe_ratio"],
        "Sortino": metrics["sortino_ratio"],
        "MDD": metrics["max_drawdown"]
    })

# 결과 테이블 생성
result_df = pd.DataFrame(result_rows)

# 보기 좋게 정렬
result_df = result_df.sort_values(["Withdraw", "Target_CVaR", "Method", "HMM"]).reset_index(drop=True)


In [13]:
result_df

,Withdraw,Target_CVaR,Method,HMM,Total Return,CAGR,Volatility,Sharpe,Sortino,MDD
0,800,0.02,6040,False,0.698218,0.065612,0.078328,0.570738,0.742981,-0.132314
1,800,0.02,CVaR_L2,False,0.800064,0.073086,0.097959,0.547163,0.647023,-0.152512
2,800,0.02,CVaR_L2,True,1.312564,0.105837,0.109793,0.775275,0.983674,-0.154994
3,800,0.03,6040,False,0.698218,0.065612,0.078328,0.570738,0.742981,-0.132314
4,800,0.03,CVaR_L2,False,1.027465,0.088515,0.114219,0.609411,0.809475,-0.186131
5,800,0.03,CVaR_L2,True,1.470062,0.114615,0.118328,0.794830,1.083918,-0.154710
6,800,0.04,6040,False,0.698218,0.065612,0.078328,0.570738,0.742981,-0.132314
7,800,0.04,CVaR_L2,False,1.025244,0.088372,0.114303,0.607890,0.807627,-0.186644
8,800,0.04,CVaR_L2,True,1.433672,0.112632,0.121730,0.761240,1.050103,-0.167329
9,800,0.05,6040,False,0.698218,0.065612,0.078328,0.570738,0.742981,-0.132314


In [33]:
result_df.to_excel('HMM(ETF_only) results.xlsx', index=False)

In [14]:
df = result_df

In [31]:
# 한글 폰트 설정 (Windows 기준 맑은 고딕 사용, 없는 경우 다른 폰트로 대체)
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False  # '-' 깨짐 방지

# 저장 경로
save_path = "C:/Users/user/Desktop/대학원/수업/2025-1/핀테크빅데이터분석심화/분석결과"

# 필터링: CVaR_L2 + HMM == True
df_filtered = df[(df["Method"] == "CVaR_L2") & (df["HMM"] == True)]

# 시각화: Withdraw 별로 line, x축은 Target_CVaR, y축은 성능지표
metrics = ["CAGR", "Sharpe", "Sortino", "MDD"]

palette = sns.color_palette("mako", n_colors=df_filtered["Withdraw"].nunique())

for metric in metrics:
    plt.figure(figsize=(8, 5))
    sns.lineplot(
        data=df_filtered,
        x="Target_CVaR",
        y=metric,
        hue="Withdraw",
        palette=palette,
        marker="o",
        err_style=None,
        ci=None
    )
    plt.title(f"{metric} vs Target_CVaR (CVaR_L2 + HMM)")
    plt.xlabel("Target CVaR")
    plt.ylabel(metric)
    plt.grid(True)
    plt.legend(title="Withdraw", loc="upper right")

    fname = f"{metric}_vs_TargetCVaR.png"
    plt.savefig(os.path.join(save_path, fname), dpi=100)
    plt.close()

In [15]:
cfg = pp.BacktestConfig(monthly_withdraw = 1_600, target_cvar = 0.03, l2_gamma = 0.1)

res_hmm_cvar = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df, use_hmm = True, method  = "CVaR_L2", cfg = cfg)
res_raw_cvar = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df, use_hmm = False, method  = "CVaR_L2", cfg = cfg)
res_6040 = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df, use_hmm = False, method  = "6040", cfg = cfg)

In [16]:
res_hmm_cvar["cum_div_income"] = res_hmm_cvar["div_income"].cumsum()
res_raw_cvar["cum_div_income"] = res_raw_cvar["div_income"].cumsum()
res_6040["cum_div_income"] = res_6040["div_income"].cumsum()

In [41]:
with pd.ExcelWriter("portfolio_backtest_outputs.xlsx", engine='openpyxl') as writer:
    res_hmm_cvar.to_excel(writer, sheet_name="res_hmm_cvar")
    res_raw_cvar.to_excel(writer, sheet_name="res_raw_cvar")
    res_6040.to_excel(writer, sheet_name="res_6040")

In [17]:
def get_metrics_df(label: str, df: pd.DataFrame, rf: pd.Series):
    return pd.DataFrame({
        "Method": [label],
        "Value_Type": ["total_value"],
        **evaluate_portfolio(df['total_value'], rf[1:len(df)], periods_per_year=12)
    }, index=[0]), pd.DataFrame({
        "Method": [label],
        "Value_Type": ["port_value"],
        **evaluate_portfolio(df['port_value'], rf[1:len(df)], periods_per_year=12)
    }, index=[0])

# 세 개 결과 각각 계산
df_list = []

for name, result in {
    "CVaR_L2 + HMM": res_hmm_cvar,
    "CVaR_L2":       res_raw_cvar,
    "60/40":         res_6040
}.items():
    df_total, df_port = get_metrics_df(name, result, monthly_rf)
    df_list.extend([df_total, df_port])

# 하나로 합치기
metrics_df = pd.concat(df_list, ignore_index=True)

# 보기 좋게 정렬
metrics_df = metrics_df[["Method", "Value_Type", "total_return", "cagr", "annualized_volatility", "sharpe_ratio", "sortino_ratio", "max_drawdown"]]

# 컬럼 이름 정리 (선택)
metrics_df.columns = ["Method", "Value", "Total Return", "CAGR", "Volatility", "Sharpe", "Sortino", "MDD"]

In [56]:
metrics_df

,Method,Value,Total Return,CAGR,Volatility,Sharpe,Sortino,MDD
0,CVaR_L2 + HMM,total_value,0.705827,0.053041,0.087080,0.429564,0.547449,-0.182130
1,CVaR_L2 + HMM,port_value,-0.286173,-0.032098,0.136435,-0.302239,-0.386078,-0.383167
2,CVaR_L2,total_value,0.697564,0.052546,0.078138,0.463440,0.576497,-0.117240
3,CVaR_L2,port_value,-0.294436,-0.033188,0.117726,-0.379685,-0.469653,-0.324775
4,60/40,total_value,0.526958,0.041813,0.055019,0.441657,0.555910,-0.082691
5,60/40,port_value,-0.465042,-0.058743,0.087059,-0.854764,-1.054448,-0.465878


In [18]:
metrics_df

,Method,Value,Total Return,CAGR,Volatility,Sharpe,Sortino,MDD
0,CVaR_L2 + HMM,total_value,1.010555,0.087422,0.109447,0.621712,0.756383,-0.145712
1,CVaR_L2 + HMM,port_value,0.210555,0.023194,0.147894,0.081682,0.107338,-0.337264
2,CVaR_L2,total_value,1.281555,0.104047,0.126547,0.674087,0.913105,-0.185954
3,CVaR_L2,port_value,0.481555,0.048301,0.160797,0.238938,0.315623,-0.297190
4,60/40,total_value,0.561728,0.054952,0.065344,0.513716,0.636484,-0.103674
5,60/40,port_value,-0.238272,-0.032132,0.090743,-0.555056,-0.687938,-0.267132


In [18]:
metrics_df.to_excel('HMM(ETF_only) result(target_cvar 0.03).xlsx', index=False)

In [20]:
save_path = "C:/Users/user/Desktop/대학원/수업/2025-1/핀테크빅데이터분석심화/분석결과"

def plot_portfolio_summary(df, label, filename, y1_lim=None, y2_lim=None):
    plt.figure(figsize=(12, 6))

    # 첫 번째 y축 (좌측)
    ax1 = sns.lineplot(x=df.index, y="total_value", data=df, label="Total Portfolio Value", color="steelblue")
    sns.lineplot(x=df.index, y="port_value", data=df, label="After Withdrawal Value", color="gold", ax=ax1)

    ax1.set_xlabel("Year")
    ax1.set_ylabel("Portfolio Value ($)")
    ax1.set_title(f"Backtest: {label}")
    ax1.tick_params(axis='x', rotation=30)
    
    if y1_lim:
        ax1.set_ylim(*y1_lim)

    # 두 번째 y축 (우측)
    ax2 = ax1.twinx()
    line2, = ax2.plot(df.index, df["cum_div_income"], color="crimson", linestyle="--")
    ax2.set_ylabel("Cumulative Dividends ($)")

    if y2_lim:
        ax2.set_ylim(*y2_lim)

    # 범례 병합
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines = lines1 + [line2]
    labels = labels1 + ["Cumulative Dividends"]
    ax1.legend(lines, labels, loc="upper left")

    plt.tight_layout()
    plt.savefig(os.path.join(save_path, filename), dpi=100)
    plt.close()

# 모든 전략에서의 y축 범위 자동 계산
all_dfs = [res_hmm_cvar, res_raw_cvar, res_6040]

# 왼쪽 y축 (포트폴리오 가치)
y1_min = min(df[["total_value", "port_value"]].min().min() for df in all_dfs)
y1_max = max(df[["total_value", "port_value"]].max().max() for df in all_dfs)

# 오른쪽 y축 (누적 배당)
y2_min = 0  # 일반적으로 배당은 음수가 없으므로 0으로 고정
y2_max = max(df["cum_div_income"].max() for df in all_dfs)

# 적용
plot_portfolio_summary(res_hmm_cvar, "CVaR_L2 + HMM", "portfolio_summary_CVaR_L2_HMM.png", (y1_min, y1_max), (y2_min, y2_max))
plot_portfolio_summary(res_raw_cvar, "CVaR_L2", "portfolio_summary_CVaR_L2.png", (y1_min, y1_max), (y2_min, y2_max))
plot_portfolio_summary(res_6040, "60/40 Strategy", "portfolio_summary_6040.png", (y1_min, y1_max), (y2_min, y2_max))

In [21]:
def plot_comparison_metric(dfs: dict, metric: str, title: str, filename: str, ylabel: str):
    plt.figure(figsize=(12, 6))
    for label, df in dfs.items():
        sns.lineplot(x=df.index, y=df[metric], label=label)

    plt.title(title)
    plt.xlabel("Year")
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, filename), dpi=100)
    plt.close()

strategy_results = {
    "CVaR_L2 + HMM": res_hmm_cvar,
    "CVaR_L2": res_raw_cvar,
    "60/40 Strategy": res_6040,
}

# 포트폴리오 총 가치 비교
plot_comparison_metric(
    dfs=strategy_results,
    metric="total_value",
    title="Total Portfolio Value Comparison",
    filename="compare_total_value.png",
    ylabel="Total Value ($)"
)

# 인출 후 가치 비교
plot_comparison_metric(
    dfs=strategy_results,
    metric="port_value",
    title="Portfolio Value After Withdrawal Comparison",
    filename="compare_port_value.png",
    ylabel="After Withdrawal Value ($)"
)

# 누적 배당금 비교
plot_comparison_metric(
    dfs=strategy_results,
    metric="cum_div_income",
    title="Cumulative Dividends Comparison",
    filename="compare_cum_dividends.png",
    ylabel="Cumulative Dividends ($)"
)

In [22]:
# 스타일 설정
sns.set(style="whitegrid")

# 필터링
hmm_df2 = hmm_df2[hmm_df2.index >= "2017-01-01"]

# 시각화 시작
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

# ─── 1단: SPY / VIX 가격 궤적 ───
ax1.plot(hmm_df2.index, hmm_df2['SPY'], label='SPY', color='tab:blue', linewidth=1.5)
ax1.set_ylabel('SPY Price', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax3 = ax1.twinx()
ax3.plot(hmm_df2.index, hmm_df2['VIX'], label='VIX', color='tab:red', alpha=0.4, linewidth=1.5)
ax3.set_ylabel('VIX Price', color='tab:red')
ax3.tick_params(axis='y', labelcolor='tab:red')

# 범례
lines1, labels1 = ax1.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
ax1.legend(lines1 + lines3, labels1 + labels3, loc="upper left")

# ─── 2단: Regime Timeline ───
regime_map = {"Bull": 2, "Neutral": 1, "Bear": 0}
res_hmm_cvar['regime_code'] = res_hmm_cvar["regime"].map(regime_map)
ax2.plot(res_hmm_cvar.index, res_hmm_cvar['regime_code'], drawstyle='steps-post', marker='o',
         markersize=3, linewidth=1, color="black")
ax2.set_yticks([0, 1, 2])
ax2.set_yticklabels(["Bear", "Neutral", "Bull"])
ax2.set_ylabel("Regime")
ax2.set_xlabel("Date")
ax2.set_title("HMM Regime Over Time", pad=6)

# 공통 x축 포맷
ax2.xaxis.set_major_locator(mdates.YearLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# 제목 및 저장
fig.suptitle("SPY / VIX Price and HMM Regime Timeline", fontsize=14)
plt.tight_layout()
plt.subplots_adjust(top=0.92)

# 저장 경로
filename = "SPY_VIX_with_HMM_Timeline.png"
plt.savefig(os.path.join(save_path, filename), dpi=100)
plt.close()

In [23]:
# 스타일
sns.set(style="whitegrid")

# 색상 지정 (가독성 높은 색)
regime_colors = {
    "Bull": "#81C784",     # 진한 연초록
    "Neutral": "#FFF176",  # 진한 노랑
    "Bear": "#E57373",     # 진한 빨강
}

# 필터링
hmm_df2 = hmm_df2[hmm_df2.index >= "2017-01-01"]

# regime 구간 추출
regime_periods = []
current_regime = res_hmm_cvar["regime"].iloc[0]
start_date = res_hmm_cvar.index[0]

for date, regime in zip(res_hmm_cvar.index[1:], res_hmm_cvar["regime"].iloc[1:]):
    if regime != current_regime:
        regime_periods.append((start_date, date, current_regime))
        current_regime = regime
        start_date = date
regime_periods.append((start_date, res_hmm_cvar.index[-1], current_regime))

# 시각화
fig, ax1 = plt.subplots(figsize=(14, 6))

# 배경 색칠
for start, end, regime in regime_periods:
    ax1.axvspan(start, end, color=regime_colors[regime], alpha=0.2)

# SPY
line1, = ax1.plot(hmm_df2.index, hmm_df2['SPY'], label='SPY', color='tab:blue', linewidth=1.5)
ax1.set_ylabel('SPY Price', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# VIX (secondary y-axis)
ax2 = ax1.twinx()
line2, = ax2.plot(hmm_df2.index, hmm_df2['VIX'], label='VIX', color='tab:red', alpha=0.4, linewidth=1.5)
ax2.set_ylabel('VIX Price', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# 범례 구성
legend_patches = [
    Patch(color='tab:blue', label='SPY'),
    Patch(color='tab:red', label='VIX'),
] + [Patch(color=c, label=reg) for reg, c in regime_colors.items()]
ax1.legend(handles=legend_patches, loc="upper left")

# x축 연도 포맷
ax1.xaxis.set_major_locator(mdates.YearLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# 제목, 저장
plt.title("SPY / VIX Price with HMM Regime Background", fontsize=14)
plt.tight_layout()

# 저장
filename = "SPY_VIX_with_HMM_Regime_Background.png"
plt.savefig(os.path.join(save_path, filename), dpi=100)
plt.close()

In [24]:
# 자산 가중치 컬럼만 추출
weight_cols = [col for col in res_hmm_cvar.columns if col.endswith("_w")]
# state별로 평균과 표준편차 계산
grouped = res_hmm_cvar.groupby("regime")[weight_cols].agg(["mean", "std"])
grouped = grouped.transpose().sort_index()

In [79]:
grouped.to_excel('HMM(ETF_only) regime별 자산 평균.xlsx', index=True)

In [19]:
save_path = "C:/Users/user/Desktop/대학원/수업/2025-1/핀테크빅데이터분석심화/분석결과"

# 스타일
sns.set(style="whitegrid")

# 공통 시각화 함수 정의
def plot_asset_weights(res_df, label, filename):
    val_cols = [c for c in res_df.columns if c.endswith("_w")]

    asset_vals = res_df[val_cols].copy()
    asset_vals.columns = [c.replace("_w", "") for c in asset_vals.columns]
    asset_vals = asset_vals.clip(lower=0, upper=None)
    weights = asset_vals.div(asset_vals.sum(axis=1), axis=0).fillna(0)

    # 팔레트 (예쁜 muted 계열)
    palette = sns.color_palette("muted", n_colors=weights.shape[1])

    # 시각화
    fig, ax = plt.subplots(figsize=(12, 6))
    weights.plot.area(ax=ax, linewidth=0, color=palette)

    ax.set_title(f"Time-Series of Asset Allocation: {label}", pad=12)
    ax.set_ylabel("Weight (%)")
    ax.set_ylim(0, 1)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.legend(loc="upper center", ncol=len(weights.columns), bbox_to_anchor=(0.5, 1.15))

    plt.tight_layout()
    plt.savefig(os.path.join(save_path, filename), dpi=100)
    plt.close()

# ───── 전략별 저장 ─────
plot_asset_weights(res_hmm_cvar, "CVaR_L2 + HMM", "weights_CVaR_L2_HMM.png")
plot_asset_weights(res_raw_cvar, "CVaR_L2", "weights_CVaR_L2.png")
plot_asset_weights(res_6040, "60/40 Strategy", "weights_6040.png")

In [40]:
def get_shared_scales(*dfs):
    max_div_yield = 0
    max_div_amount = 0
    max_asset_div = 0

    for df in dfs:
        df = df.copy()
        df["dividends"] = df["cum_div_income"].diff().fillna(df["cum_div_income"].iloc[0])
        df["year"] = df.index.year
        div_by_year = df["dividends"].groupby(df.index.year).sum()
        annual_yield = (div_by_year / 200000) * 100
        max_div_yield = max(max_div_yield, annual_yield.max())
        max_div_amount = max(max_div_amount, div_by_year.max())

        div_cols = [c for c in df.columns if c.endswith("_div")]
        div_asset = df.groupby("year")[div_cols].sum()
        max_asset_div = max(max_asset_div, div_asset.values.max())

    return max_div_amount, max_div_yield, max_asset_div

def plot_dividend_analysis(df: pd.DataFrame, name: str, out_dir: str,
                           max_div_amt: float, max_div_yield: float, max_asset_div: float):
    res = df.copy()
    res["dividends"] = res["cum_div_income"].diff().fillna(res["cum_div_income"].iloc[0])
    res["year"] = res.index.year
    dividends_by_year = res["dividends"].groupby(res.index.year).sum()
    annual_yields = (dividends_by_year / 200000) * 100

    plot_years = annual_yields.index[annual_yields.index < 2025]
    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.bar(plot_years, dividends_by_year.loc[plot_years].values,
            alpha=0.3, color="skyblue", label="dividends by year")
    ax1.set_ylabel("Dividends by Year ($)")
    ax1.set_ylim(0, max_div_amt * 1.1)
    ax1.set_title(f"{name} - Annual Dividend Yield")

    ax2 = ax1.twinx()
    y_vals = annual_yields.loc[plot_years].values
    ax2.plot(plot_years, y_vals, color="navy", linewidth=2, marker='o', label="dividend yield")
    ax2.set_ylabel("Dividend yield (%)")
    ax2.set_ylim(0, max_div_yield * 1.2)
    ax2.set_yticks(range(0, int(max_div_yield)+2))
    ax2.yaxis.set_major_formatter(mtick.PercentFormatter())

    for x, y in zip(plot_years, y_vals):
        ax2.annotate(f"{y:.2f}%", (x, y), textcoords="offset points", xytext=(0, 10),
                     ha='center', fontsize=9, color="black")

    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{name}_annual_dividend_yield.png"))
    plt.close(fig)

    # Plot 2: Dividends by Asset
    div_cols = [c for c in res.columns if c.endswith("_div")]
    div_by_asset_year = res.groupby("year")[div_cols].sum()
    weight_cols = [c for c in res.columns if c.endswith("_w")]
    avg_weights = res.groupby("year")[weight_cols].mean() * 100
    avg_weights_table = avg_weights.T
    avg_weights_table.index = [c.replace("_w", "") for c in avg_weights_table.index]

    common_years = div_by_asset_year.index.intersection(avg_weights_table.columns)
    div_by_asset_year = div_by_asset_year.loc[common_years]
    avg_weights_table = avg_weights_table[common_years]

    fig, ax = plt.subplots(figsize=(12, 6))
    div_by_asset_year.plot(kind="bar", ax=ax)
    ax.set_title(f"{name} - Yearly Dividends by Asset")
    ax.set_ylabel("Dividends by Year ($)")
    ax.set_ylim(0, max_asset_div * 1.1)
    ax.set_xlabel("Year")
    ax.set_xticks(range(len(div_by_asset_year.index)))
    ax.set_xticklabels(div_by_asset_year.index, rotation=0)
    ax.grid(True)

    table_data = avg_weights_table.round(1).astype(str) + "%"
    table = plt.table(
        cellText=table_data.values,
        rowLabels=table_data.index,
        colLabels=table_data.columns,
        cellLoc='center',
        rowLoc='center',
        loc='bottom',
        bbox=[0.0, -0.85, 1.0, 0.5]
    )

    plt.subplots_adjust(bottom=0.35)
    fig.tight_layout()
    fig.savefig(os.path.join(out_dir, f"{name}_dividends_by_asset.png"))
    plt.close(fig)

# 세 가지 결과 데이터프레임에 대해 저장
max_amt, max_yield, max_asset = get_shared_scales(res_hmm_cvar, res_raw_cvar, res_6040)

plot_dividend_analysis(res_hmm_cvar, "res_hmm_cvar", save_path, max_amt, max_yield, max_asset)
plot_dividend_analysis(res_raw_cvar, "res_raw_cvar", save_path, max_amt, max_yield, max_asset)
plot_dividend_analysis(res_6040, "res_6040", save_path, max_amt, max_yield, max_asset)

In [75]:
def calc_total_avg_div_yield(df: pd.DataFrame, principal: float = 200_000) -> float:
    df = df.copy()
    df["dividends"] = df["cum_div_income"].diff().fillna(df["cum_div_income"].iloc[0])
    df["year"] = df.index.year

    last_year = df["year"].max()
    full_years = df["year"].unique()
    full_years = [y for y in full_years if y != last_year]

    dividends_by_year = df.groupby("year")["dividends"].sum()
    total_div = dividends_by_year.loc[full_years].sum()
    n_years = len(full_years)

    avg_yield = (total_div / principal) / n_years * 100
    return avg_yield

# 전략 이름과 결과 DataFrame 매핑
strategies = {
    "HMM_CVaR_L2": res_hmm_cvar,
    "Raw_CVaR_L2": res_raw_cvar,
    "60/40":       res_6040,
}

# 연평균 분배율 계산 결과 저장
avg_yield_data = {
    name: calc_total_avg_div_yield(df)
    for name, df in strategies.items()
}

# 데이터프레임으로 변환
avg_yield_df = pd.DataFrame.from_dict(avg_yield_data, orient='index', columns=["Avg Dividend Yield (%)"])
avg_yield_df.index.name = "Strategy"

'\ndef calc_avg_of_annual_yields(df: pd.DataFrame, principal: float = 200_000) -> float:\n    df = df.copy()\n    df["dividends"] = df["cum_div_income"].diff().fillna(df["cum_div_income"].iloc[0])\n    df["year"] = df.index.year\n\n    last_year = df["year"].max()\n    dividends_by_year = df.groupby("year")["dividends"].sum()\n    \n    # 마지막 해 제외\n    dividends_by_year = dividends_by_year.loc[dividends_by_year.index != last_year]\n\n    annual_yields = (dividends_by_year / principal) * 100\n    return annual_yields.mean()\n\n\navg_yield_hmm_cvar = calc_avg_of_annual_yields(res_hmm_cvar)\navg_yield_raw_cvar = calc_avg_of_annual_yields(res_raw_cvar)\navg_yield_6040     = calc_avg_of_annual_yields(res_6040)\n\nprint("HMM CVaR_L2 연도별 평균 배당 수익률: {:.2f}%".format(avg_yield_hmm_cvar))\nprint("Raw CVaR_L2 연도별 평균 배당 수익률: {:.2f}%".format(avg_yield_raw_cvar))\nprint("60/40 연도별 평균 배당 수익률: {:.2f}%".format(avg_yield_6040))\n'

In [77]:
avg_yield_df.to_excel('avg_yield_df.xlsx', index=True)

#### 리밸런싱 주기별

In [81]:
# 전략 설정
methods = {
    "HMM_CVaR_L2": {"use_hmm": True, "method": "CVaR_L2"},
    "Raw_CVaR_L2": {"use_hmm": False, "method": "CVaR_L2"},
    "6040":        {"use_hmm": False, "method": "6040"},
}

rebalance_options = ["monthly", "quarterly", "semiannual", "annual"]
withdrawals = [800, 1600]

# 각 조합의 결과를 저장할 딕셔너리
result_dfs = {}

for strategy_name, params in methods.items():
    for withdraw in withdrawals:
        for freq in rebalance_options:
            cfg = pp.BacktestConfig(
                monthly_withdraw=withdraw,
                target_cvar=0.03,
                rebalance_freq=freq
            )

            # 백테스트 실행
            res = pp.backtest(
                price_df, adj_price_df, dividend_df, hmm_df,
                use_hmm=params["use_hmm"],
                method=params["method"],
                cfg=cfg
            )

            # 이름 예: res_HMM_CVaR_L2_W1600_monthly
            varname = f"res_{strategy_name}_W{withdraw}_{freq}"
            result_dfs[varname] = res

In [82]:
# 모든 조합 명시적 추출
res_HMM_CVaR_L2_W800_monthly     = result_dfs["res_HMM_CVaR_L2_W800_monthly"]
res_HMM_CVaR_L2_W800_quarterly   = result_dfs["res_HMM_CVaR_L2_W800_quarterly"]
res_HMM_CVaR_L2_W800_semiannual  = result_dfs["res_HMM_CVaR_L2_W800_semiannual"]
res_HMM_CVaR_L2_W800_annual      = result_dfs["res_HMM_CVaR_L2_W800_annual"]

res_HMM_CVaR_L2_W1600_monthly    = result_dfs["res_HMM_CVaR_L2_W1600_monthly"]
res_HMM_CVaR_L2_W1600_quarterly  = result_dfs["res_HMM_CVaR_L2_W1600_quarterly"]
res_HMM_CVaR_L2_W1600_semiannual = result_dfs["res_HMM_CVaR_L2_W1600_semiannual"]
res_HMM_CVaR_L2_W1600_annual     = result_dfs["res_HMM_CVaR_L2_W1600_annual"]

res_Raw_CVaR_L2_W800_monthly     = result_dfs["res_Raw_CVaR_L2_W800_monthly"]
res_Raw_CVaR_L2_W800_quarterly   = result_dfs["res_Raw_CVaR_L2_W800_quarterly"]
res_Raw_CVaR_L2_W800_semiannual  = result_dfs["res_Raw_CVaR_L2_W800_semiannual"]
res_Raw_CVaR_L2_W800_annual      = result_dfs["res_Raw_CVaR_L2_W800_annual"]

res_Raw_CVaR_L2_W1600_monthly    = result_dfs["res_Raw_CVaR_L2_W1600_monthly"]
res_Raw_CVaR_L2_W1600_quarterly  = result_dfs["res_Raw_CVaR_L2_W1600_quarterly"]
res_Raw_CVaR_L2_W1600_semiannual = result_dfs["res_Raw_CVaR_L2_W1600_semiannual"]
res_Raw_CVaR_L2_W1600_annual     = result_dfs["res_Raw_CVaR_L2_W1600_annual"]

res_6040_W800_monthly            = result_dfs["res_6040_W800_monthly"]
res_6040_W800_quarterly          = result_dfs["res_6040_W800_quarterly"]
res_6040_W800_semiannual         = result_dfs["res_6040_W800_semiannual"]
res_6040_W800_annual             = result_dfs["res_6040_W800_annual"]

res_6040_W1600_monthly           = result_dfs["res_6040_W1600_monthly"]
res_6040_W1600_quarterly         = result_dfs["res_6040_W1600_quarterly"]
res_6040_W1600_semiannual        = result_dfs["res_6040_W1600_semiannual"]
res_6040_W1600_annual            = result_dfs["res_6040_W1600_annual"]

In [91]:
# 저장 경로 설정
excel_path = os.path.join(save_path, "all_backtest_results.xlsx")

# 엑셀 라이터 열기
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    for name, df in result_dfs.items():
        # 시트 이름은 너무 길면 엑셀에서 오류 → 최대 31자 제한
        sheet_name = name.replace("res_", "")[:31]
        df.to_excel(writer, sheet_name=sheet_name)

In [85]:
# 평가 함수 그대로 사용
def get_metrics_df(label: str, df: pd.DataFrame, rf: pd.Series):
    return pd.DataFrame({
        "Method": [label],
        "Value_Type": ["total_value"],
        **evaluate_portfolio(df['total_value'], rf[1:len(df)], periods_per_year=12)
    }, index=[0]), pd.DataFrame({
        "Method": [label],
        "Value_Type": ["port_value"],
        **evaluate_portfolio(df['port_value'], rf[1:len(df)], periods_per_year=12)
    }, index=[0])

# 전략, 인출금, 리밸런싱 조합 정의
strategies = ["HMM_CVaR_L2", "Raw_CVaR_L2", "6040"]
withdrawals = [800, 1600]
rebalances = ["monthly", "quarterly", "semiannual", "annual"]

# 결과 보관 리스트
df_list = []

for strategy in strategies:
    for withdraw in withdrawals:
        for freq in rebalances:
            varname = f"res_{strategy}_W{withdraw}_{freq}"
            df = result_dfs[varname]

            label = f"{strategy} | W{withdraw} | {freq}"
            df_total, df_port = get_metrics_df(label, df, monthly_rf)
            df_list.extend([df_total, df_port])

# 하나의 DataFrame으로 정리
metrics_df = pd.concat(df_list, ignore_index=True)

# 보기 좋게 컬럼 정리
metrics_df = metrics_df[[
    "Method", "Value_Type", "total_return", "cagr",
    "annualized_volatility", "sharpe_ratio",
    "sortino_ratio", "max_drawdown"
]]

metrics_df.columns = [
    "Method", "Value", "Total Return", "CAGR",
    "Volatility", "Sharpe", "Sortino", "MDD"
]

In [87]:
metrics_df.to_excel('HMM(ETF_only) result(target_cvar 0.03), 리밸런싱 주기.xlsx', index=False)

In [90]:
# 1. 누적 배당금 컬럼 추가
for key in [
    "res_HMM_CVaR_L2_W1600_monthly",
    "res_HMM_CVaR_L2_W1600_quarterly",
    "res_HMM_CVaR_L2_W1600_semiannual",
    "res_HMM_CVaR_L2_W1600_annual",
    "res_Raw_CVaR_L2_W1600_monthly",
    "res_Raw_CVaR_L2_W1600_quarterly",
    "res_Raw_CVaR_L2_W1600_semiannual",
    "res_Raw_CVaR_L2_W1600_annual",
    "res_6040_W1600_monthly",
    "res_6040_W1600_quarterly",
    "res_6040_W1600_semiannual",
    "res_6040_W1600_annual"
]:
    result_dfs[key]["cum_div_income"] = result_dfs[key]["div_income"].cumsum()

# 2. 시각화 함수
def plot_rebalance_comparison(base_key: str, title: str, metric: str, ylabel: str, filename: str):
    plt.figure(figsize=(12, 6))
    for freq in ["monthly", "quarterly", "semiannual", "annual"]:
        key = f"{base_key}_{freq}"
        df = result_dfs[key]
        label = freq.capitalize()
        sns.lineplot(x=df.index, y=df[metric], label=label)

    plt.title(title)
    plt.xlabel("Year")
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.legend(title="Rebalance")
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, filename), dpi=100)
    plt.close()

# 3. 전략별 시각화 호출
plot_rebalance_comparison(
    base_key="res_HMM_CVaR_L2_W1600",
    title="HMM CVaR_L2 - Total Portfolio Value by Rebalancing",
    metric="total_value",
    ylabel="Total Value ($)",
    filename="hmm_cvar_total_value.png"
)

plot_rebalance_comparison(
    base_key="res_HMM_CVaR_L2_W1600",
    title="HMM CVaR_L2 - After Withdrawal Portfolio Value",
    metric="port_value",
    ylabel="After Withdrawal Value ($)",
    filename="hmm_cvar_port_value.png"
)

plot_rebalance_comparison(
    base_key="res_HMM_CVaR_L2_W1600",
    title="HMM CVaR_L2 - Cumulative Dividends",
    metric="cum_div_income",
    ylabel="Cumulative Dividends ($)",
    filename="hmm_cvar_cum_dividends.png"
)

plot_rebalance_comparison(
    base_key="res_Raw_CVaR_L2_W1600",
    title="Raw CVaR_L2 - Total Portfolio Value by Rebalancing",
    metric="total_value",
    ylabel="Total Value ($)",
    filename="raw_cvar_total_value.png"
)

plot_rebalance_comparison(
    base_key="res_Raw_CVaR_L2_W1600",
    title="Raw CVaR_L2 - After Withdrawal Portfolio Value",
    metric="port_value",
    ylabel="After Withdrawal Value ($)",
    filename="raw_cvar_port_value.png"
)

plot_rebalance_comparison(
    base_key="res_Raw_CVaR_L2_W1600",
    title="Raw CVaR_L2 - Cumulative Dividends",
    metric="cum_div_income",
    ylabel="Cumulative Dividends ($)",
    filename="raw_cvar_cum_dividends.png"
)

plot_rebalance_comparison(
    base_key="res_6040_W1600",
    title="60/40 - Total Portfolio Value by Rebalancing",
    metric="total_value",
    ylabel="Total Value ($)",
    filename="6040_total_value.png"
)

plot_rebalance_comparison(
    base_key="res_6040_W1600",
    title="60/40 - After Withdrawal Portfolio Value",
    metric="port_value",
    ylabel="After Withdrawal Value ($)",
    filename="6040_port_value.png"
)

plot_rebalance_comparison(
    base_key="res_6040_W1600",
    title="60/40 - Cumulative Dividends",
    metric="cum_div_income",
    ylabel="Cumulative Dividends ($)",
    filename="6040_cum_dividends.png"
)

In [92]:
# 1. 누적 배당금 추가 (W800)
for key in [
    "res_HMM_CVaR_L2_W800_monthly",
    "res_HMM_CVaR_L2_W800_quarterly",
    "res_HMM_CVaR_L2_W800_semiannual",
    "res_HMM_CVaR_L2_W800_annual",
    "res_Raw_CVaR_L2_W800_monthly",
    "res_Raw_CVaR_L2_W800_quarterly",
    "res_Raw_CVaR_L2_W800_semiannual",
    "res_Raw_CVaR_L2_W800_annual",
    "res_6040_W800_monthly",
    "res_6040_W800_quarterly",
    "res_6040_W800_semiannual",
    "res_6040_W800_annual"
]:
    result_dfs[key]["cum_div_income"] = result_dfs[key]["div_income"].cumsum()

# 2. 전략별 시각화 호출 (W800)
plot_rebalance_comparison(
    base_key="res_HMM_CVaR_L2_W800",
    title="HMM CVaR_L2 - Total Portfolio Value by Rebalancing (W800)",
    metric="total_value",
    ylabel="Total Value ($)",
    filename="hmm_cvar_total_value_W800.png"
)

plot_rebalance_comparison(
    base_key="res_HMM_CVaR_L2_W800",
    title="HMM CVaR_L2 - After Withdrawal Portfolio Value (W800)",
    metric="port_value",
    ylabel="After Withdrawal Value ($)",
    filename="hmm_cvar_port_value_W800.png"
)

plot_rebalance_comparison(
    base_key="res_HMM_CVaR_L2_W800",
    title="HMM CVaR_L2 - Cumulative Dividends (W800)",
    metric="cum_div_income",
    ylabel="Cumulative Dividends ($)",
    filename="hmm_cvar_cum_dividends_W800.png"
)

plot_rebalance_comparison(
    base_key="res_Raw_CVaR_L2_W800",
    title="Raw CVaR_L2 - Total Portfolio Value by Rebalancing (W800)",
    metric="total_value",
    ylabel="Total Value ($)",
    filename="raw_cvar_total_value_W800.png"
)

plot_rebalance_comparison(
    base_key="res_Raw_CVaR_L2_W800",
    title="Raw CVaR_L2 - After Withdrawal Portfolio Value (W800)",
    metric="port_value",
    ylabel="After Withdrawal Value ($)",
    filename="raw_cvar_port_value_W800.png"
)

plot_rebalance_comparison(
    base_key="res_Raw_CVaR_L2_W800",
    title="Raw CVaR_L2 - Cumulative Dividends (W800)",
    metric="cum_div_income",
    ylabel="Cumulative Dividends ($)",
    filename="raw_cvar_cum_dividends_W800.png"
)

plot_rebalance_comparison(
    base_key="res_6040_W800",
    title="60/40 - Total Portfolio Value by Rebalancing (W800)",
    metric="total_value",
    ylabel="Total Value ($)",
    filename="6040_total_value_W800.png"
)

plot_rebalance_comparison(
    base_key="res_6040_W800",
    title="60/40 - After Withdrawal Portfolio Value (W800)",
    metric="port_value",
    ylabel="After Withdrawal Value ($)",
    filename="6040_port_value_W800.png"
)

plot_rebalance_comparison(
    base_key="res_6040_W800",
    title="60/40 - Cumulative Dividends (W800)",
    metric="cum_div_income",
    ylabel="Cumulative Dividends ($)",
    filename="6040_cum_dividends_W800.png"
)

In [93]:
# 스타일 설정
sns.set(style="whitegrid")

# 공통 시각화 함수 정의 (동일)
def plot_asset_weights(res_df, label, filename):
    val_cols = [c for c in res_df.columns if c.endswith("_w")]

    asset_vals = res_df[val_cols].copy()
    asset_vals.columns = [c.replace("_w", "") for c in asset_vals.columns]
    asset_vals = asset_vals.clip(lower=0, upper=None)
    weights = asset_vals.div(asset_vals.sum(axis=1), axis=0).fillna(0)

    # 예쁜 색 팔레트
    palette = sns.color_palette("muted", n_colors=weights.shape[1])

    # 시각화
    fig, ax = plt.subplots(figsize=(12, 6))
    weights.plot.area(ax=ax, linewidth=0, color=palette)

    ax.set_title(f"Time-Series of Asset Allocation: {label}", pad=12)
    ax.set_ylabel("Weight (%)")
    ax.set_ylim(0, 1)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    ax.legend(loc="upper center", ncol=len(weights.columns), bbox_to_anchor=(0.5, 1.15))

    plt.tight_layout()
    plt.savefig(os.path.join(save_path, filename), dpi=100)
    plt.close()

# 전략 가독성용 딕셔너리 (이름 정리)
strategy_label_map = {
    "HMM_CVaR_L2": "CVaR_L2 + HMM",
    "Raw_CVaR_L2": "CVaR_L2",
    "6040": "60/40 Strategy"
}

# 반복 실행
for key, df in result_dfs.items():
    parts = key.replace("res_", "").split("_")
    strategy = "_".join(parts[:2]) if parts[0] == "6040" else "_".join(parts[:3])
    withdraw = parts[-2]  # W800 or W1600
    freq = parts[-1]      # monthly, quarterly, etc

    label = f"{strategy_label_map.get(strategy, strategy)} | {withdraw.upper()} | {freq.capitalize()}"
    filename = f"weights_{strategy}_{withdraw}_{freq}.png"

    plot_asset_weights(df, label, filename)

#### 고갈시점

In [94]:
cfg = pp.BacktestConfig(initial_cash = 100_000, monthly_withdraw = 1_600, target_cvar = 0.03)

res_hmm_cvar = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df, use_hmm = True, method  = "CVaR_L2", cfg = cfg)
res_raw_cvar = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df, use_hmm = False, method  = "CVaR_L2", cfg = cfg)
res_6040 = pp.backtest(price_df, adj_price_df, dividend_df, hmm_df, use_hmm = False, method  = "6040", cfg = cfg)

In [100]:
res_hmm_cvar["cum_div_income"] = res_hmm_cvar["div_income"].cumsum()
res_raw_cvar["cum_div_income"] = res_raw_cvar["div_income"].cumsum()
res_6040["cum_div_income"] = res_6040["div_income"].cumsum()

In [101]:
with pd.ExcelWriter("portfolio_backtest_outputs.xlsx", engine='openpyxl') as writer:
    res_hmm_cvar.to_excel(writer, sheet_name="res_hmm_cvar")
    res_raw_cvar.to_excel(writer, sheet_name="res_raw_cvar")
    res_6040.to_excel(writer, sheet_name="res_6040")

In [98]:
def detect_depletion_date(df: pd.DataFrame) -> str:
    """port_value가 0 이하가 된 최초 날짜. 없으면 'None' 반환"""
    depleted = df[df["port_value"] <= 0]
    if not depleted.empty:
        return depleted.index[0].strftime("%Y-%m")
    else:
        return "None"

def get_metrics_df(label: str, df: pd.DataFrame, rf: pd.Series):
    depletion = detect_depletion_date(df)

    df_total = pd.DataFrame({
        "Method": [label],
        "Value_Type": ["total_value"],
        **evaluate_portfolio(df['total_value'], rf[1:len(df)], periods_per_year=12),
        "Depletion_Date": [depletion]
    })

    df_port = pd.DataFrame({
        "Method": [label],
        "Value_Type": ["port_value"],
        **evaluate_portfolio(df['port_value'], rf[1:len(df)], periods_per_year=12),
        "Depletion_Date": [depletion]
    })

    return df_total, df_port

# 평가 대상 전략들
strategy_results = {
    "CVaR_L2 + HMM": res_hmm_cvar,
    "CVaR_L2": res_raw_cvar,
    "60/40 Strategy": res_6040,
}

# 모든 메트릭 수집
df_list = []
for name, result in strategy_results.items():
    df_total, df_port = get_metrics_df(name, result, monthly_rf)
    df_list.extend([df_total, df_port])

# 합치기
metrics_df = pd.concat(df_list, ignore_index=True)

# 컬럼 정리
metrics_df = metrics_df[[
    "Method", "Value_Type", "total_return", "cagr",
    "annualized_volatility", "sharpe_ratio", "sortino_ratio", "max_drawdown", "Depletion_Date"
]]

metrics_df.columns = [
    "Method", "Value", "Total Return", "CAGR",
    "Volatility", "Sharpe", "Sortino", "MDD", "Depletion Date"
]

In [102]:
metrics_df.to_excel('metrics_df.xlsx', index=True)

In [103]:
plot_comparison_metric(
    dfs=strategy_results,
    metric="total_value",
    title="Total Portfolio Value Comparison",
    filename="compare_total_value.png",
    ylabel="Total Value ($)"
)

plot_comparison_metric(
    dfs=strategy_results,
    metric="port_value",
    title="Portfolio Value After Withdrawal Comparison",
    filename="compare_port_value.png",
    ylabel="After Withdrawal Value ($)"
)

plot_comparison_metric(
    dfs=strategy_results,
    metric="cum_div_income",
    title="Cumulative Dividends Comparison",
    filename="compare_cum_dividends.png",
    ylabel="Cumulative Dividends ($)"
)